In [77]:
import tensorflow as tf
import keras 
from keras.applications import VGG16
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers

In [78]:
conv_base = VGG16(weights='imagenet', include_top = False, input_shape=(300,300,3))

58892288/58889256 [==============================] - 6s 0us/step


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

In [79]:
X = np.load("gdrive/My Drive/pcb/xtrain.npy")
y = np.load("gdrive/My Drive/pcb/ytrain.npy")

In [80]:
conv_base.predict(X[0].reshape(1,300,300,3)).shape

(1, 9, 9, 512)

In [ ]:
# Training set has 472 samples

Training with k-fold Validation

In [81]:
X.shape, (y == 0).sum(), (y == 1).sum()

((472, 300, 300, 3), 236, 236)

In [82]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagen.fit(X)

In [83]:
nos = X.shape[0]
bs = int(nos/8)

numit = NumpyArrayIterator(
    X, y, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

samples = np.zeros(shape=(nos,9,9,512))
labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    samples[i*bs:(i+1)*bs] = f
    labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)
(59, 9, 9, 512)


In [87]:
samples = samples.reshape(nos,9*9*512)

In [88]:
samples.shape, labels.shape

((472, 41472), (472,))

In [ ]:
# To save memory
del X
del y

In [90]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)

accuracies = []
f = 1

for train_index, test_index in skf.split(samples, labels):
        X_train = samples[train_index]
        X_test = samples[test_index]
        y_train = labels[train_index]
        y_test = labels[test_index]
      
        model = models.Sequential()
        model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*512))
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(1, activation = 'sigmoid'))
        model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
        my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]

        print('Fold {}'.format(f))
        history = model.fit(X_train, y_train, epochs = 100, batch_size = 20, validation_data = (X_test, y_test), shuffle = True, callbacks = my_callbacks)

        test_preds = model.predict(X_test, batch_size = 20)
        test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

        accuracies.append([confusion_matrix(y_test,test_predclass)[0][0]/(y_test == 0.0).sum(), confusion_matrix(y_test,test_predclass)[1][1]/(y_test == 1.0).sum()])
        f+=1


Fold 1
Train on 413 samples, validate on 59 samples
Epoch 1/100
413/413 [==============================] - 1s 1ms/step - loss: 2.0245 - accuracy: 0.5932 - val_loss: 0.7656 - val_accuracy: 0.6441
Epoch 2/100
413/413 [==============================] - 0s 972us/step - loss: 0.4952 - accuracy: 0.7433 - val_loss: 0.5744 - val_accuracy: 0.7797
Epoch 3/100
413/413 [==============================] - 0s 957us/step - loss: 0.3565 - accuracy: 0.8523 - val_loss: 0.5137 - val_accuracy: 0.7458
Epoch 4/100
413/413 [==============================] - 0s 979us/step - loss: 0.2810 - accuracy: 0.8910 - val_loss: 0.5105 - val_accuracy: 0.7966
Epoch 5/100
413/413 [==============================] - 0s 961us/step - loss: 0.2528 - accuracy: 0.8935 - val_loss: 0.4763 - val_accuracy: 0.8136
Epoch 6/100
413/413 [==============================] - 0s 980us/step - loss: 0.1886 - accuracy: 0.9298 - val_loss: 0.4540 - val_accuracy: 0.8305
Epoch 7/100
413/413 [==============================] - 0s 965us/step - loss: 0.1

In [91]:
zero_acc = 0
one_acc = 0

for i in accuracies:
  zero_acc+= i[0]
  one_acc+=i[1]

zero_acc/=8
one_acc/=8

print(zero_acc, one_acc)

0.7895114942528736 0.8475574712643679


Training on whole Training Dataset

In [92]:
model = models.Sequential()
model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]


In [93]:
xtest = np.load('gdrive/My Drive/pcb/xtest.npy')
ytest = np.load('gdrive/My Drive/pcb/ytest.npy')


In [94]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

In [95]:
datagentest.fit(xtest)

In [99]:
testnos = ytest.shape[0]

numittest = NumpyArrayIterator(
    xtest, ytest, datagentest, batch_size=testnos, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


test_samples = np.zeros(shape=(testnos,9,9,512))
test_labels = np.zeros(shape=(testnos))
for samples_batch,labels_batch in numittest:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    test_samples[::] = g
    test_labels[:] = labels_batch
    break

(552, 9, 9, 512)


In [100]:
test_samples = test_samples.reshape(testnos,9*9*512)

In [101]:
# To save memory
del xtest
del ytest

In [102]:
historytest = model.fit(samples, labels, epochs = 100, batch_size = 20, validation_data = (test_samples, test_labels), shuffle = True,  callbacks = my_callbacks)

Train on 472 samples, validate on 552 samples
Epoch 1/100
472/472 [==============================] - 1s 2ms/step - loss: 1.6124 - accuracy: 0.6992 - val_loss: 1.2413 - val_accuracy: 0.6051
Epoch 2/100
472/472 [==============================] - 1s 1ms/step - loss: 0.4857 - accuracy: 0.8220 - val_loss: 0.7223 - val_accuracy: 0.7319
Epoch 3/100
472/472 [==============================] - 1s 1ms/step - loss: 0.2787 - accuracy: 0.8835 - val_loss: 0.8662 - val_accuracy: 0.6812
Epoch 4/100
472/472 [==============================] - 1s 1ms/step - loss: 0.1825 - accuracy: 0.9301 - val_loss: 0.4512 - val_accuracy: 0.8134
Epoch 5/100
472/472 [==============================] - 1s 1ms/step - loss: 0.1377 - accuracy: 0.9576 - val_loss: 0.6727 - val_accuracy: 0.7627
Epoch 6/100
472/472 [==============================] - 1s 1ms/step - loss: 0.1281 - accuracy: 0.9513 - val_loss: 1.1811 - val_accuracy: 0.6250
Epoch 7/100
472/472 [==============================] - 1s 1ms/step - loss: 0.1067 - accuracy: 0.

In [103]:
test_preds = model.predict(test_samples, batch_size = 20)

In [104]:
test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

In [109]:
confusion_matrix(test_labels,test_predclass)[0][0]/(test_labels == 0.0).sum(), confusion_matrix(test_labels,test_predclass)[1][1]/(test_labels == 1.0).sum()

(0.711864406779661, 0.8255578093306288)

In [ ]:
# Saving model

# model_json = model.to_json()
# with open("gdrive/My Drive/models/VGG19_12.json", "w") as json_file:
#     json_file.write(model_json)

In [ ]:
# model.save_weights("gdrive/My Drive/models/VGG19_12.h5")
# print("Saved model to disk")

In [ ]:
# Saving and loading models
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/

TypeError: ignored